In [1]:
import numpy as np
import tensorflow as tf
sess = tf.InteractiveSession()
print('module loaded')
print(tf.__version__)
#https://www.tensorflow.org/get_started/mnist/pros

module loaded
1.0.1


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder("float", shape=[None, 784])  #입력
y_ = tf.placeholder("float", shape=[None,10]) #출력

In [4]:
W = tf.Variable(tf.zeros([784,10])) #웨이트
b = tf.Variable(tf.zeros([10])) #바이어스

In [5]:
sess.run(tf.global_variables_initializer()) #이니셜라이즈 

In [6]:
y = tf.matmul(x,W) + b #y = wx + b
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)) # loss = cross entropy

In [7]:
#train model
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy) #GD, lr = 0.5
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [8]:
#evaluate model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9206


In [9]:
# InternalError: Dst tensor is not initialized. > 메모리 문제. 백그라운드 실행되는것들 꺼주면 됨
#http://pythonkim.tistory.com/56

In [12]:
# convolution & pooling
def conv2d(x, W): #2d convolution 
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') # SAME > input = output

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


# initialize variable
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1) #정규분포에서 일정한 값을 잘라냄
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial) #상수

In [13]:
W_conv1 = weight_variable([5, 5, 1, 32]) # 5x5x1channel > ? x ? x 32
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # w*(conv) x + b
h_pool1 = max_pool_2x2(h_conv1)
# 1번째 conv -> pool

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
#두번째 히든 레이어로 가기 위한 weight/bias 이번에는 5 x 5 x 32 > ? x ? x 64

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
# 2번째 conv -> pool

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
# fully connected layer 7x7x64 > 1024변환을 위한 weight/bias

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# cnn -> fc 

keep_prob = tf.placeholder(tf.float32) #placeholder
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) #드랍아웃. tf에 다 구현되어있음. 

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
#마지막 1024 > 10바꾸려는 weight + bias

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
#결과다. 

In [14]:
#train & eval
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)) #logits이라는게... 최종 classify전의 결과 같은데.. 아니면 softmax같은거라던가
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) #adam optimizer. 15년 논문이라는데.. 성능이 SGD보다 훨씬 좋으나 어렵군 
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1)) #잘 맞은애들만 트루로 나올듯
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #잘 맞은애들을 float으로 캐스트해서 평균냄 
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5}) #옵티마이저, 로스펑션 등등 설정해둔걸 돌리면 됨.

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.18
step 100, training accuracy 0.88
step 200, training accuracy 0.9
step 300, training accuracy 0.94
step 400, training accuracy 0.94
step 500, training accuracy 0.98
step 600, training accuracy 0.98
step 700, training accuracy 0.98
step 800, training accuracy 0.94
step 900, training accuracy 0.98
step 1000, training accuracy 0.96
step 1100, training accuracy 1
step 1200, training accuracy 0.96
step 1300, training accuracy 0.96
step 1400, training accuracy 0.98
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.96
step 1900, training accuracy 0.96
step 2000, training accuracy 0.94
step 2100, training accuracy 0.94
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, traini